In [ ]:
# enable if running in colab
# !pip install git+https://github.com/patrickjohncyh/nlp_ml_api

In [ ]:
# enable if running in colab
# !mkdir datasets
# !wget https://github.com/patrickjohncyh/nlp_ml_api/raw/master/datasets/corona_nlp.zip
# !unzip corona_nlp.zip -d ./datasets/

In [ ]:
import os
import time
import requests
import pandas as pd
from nlp_ml_api.utils.model_utils import  get_available_models, deploy_model
from nlp_ml_api.datasets import C19TwitterSentimentDataset
from nlp_ml_api.Modeler import Modeler

In [ ]:
df_train = pd.read_csv(os.path.join('datasets/corona_nlp', 'Corona_NLP_train.csv'), encoding="ISO-8859-1")
df_test = pd.read_csv(os.path.join('datasets/corona_nlp', 'Corona_NLP_test.csv'), encoding="ISO-8859-1")

In [ ]:
# transform data to same format as _requirements_
training_data =[{'string':row['OriginalTweet'], 'labels':row['Sentiment']}
                  for _,row in  df_train[['OriginalTweet', 'Sentiment']].iterrows()]

testing_data =[{'string':row['OriginalTweet'], 'labels':row['Sentiment']}
                  for _,row in  df_test[['OriginalTweet', 'Sentiment']].iterrows()]

In [ ]:
# load dataset
dataset = C19TwitterSentimentDataset.from_list(
    data_list_train=training_data,
    data_list_test=testing_data
)

In [ ]:
# list available models
get_available_models()

In [ ]:
# load Modeler with model and dataset
# Change to TransformerModel if running on colab!
modeler = Modeler('TFIDFModel', dataset)

In [ ]:
# train model
modeler.train()

In [ ]:
# evaluate on test set (split='train') to evaluate on train
eval_result = modeler.evaluate(split='test')

In [ ]:
model = modeler.model

In [ ]:
model.predict('Quarantine Day 5: I forgot how the outside world looks like anymore.')

In [ ]:
model.predict('Quarantine Day 129438: I am tired.')

### Localhost Deployment
We can deploy onto localhost and invoke model via

`0.0.0.0/endpoint/?query=myquery`

(Do not run if on Colab)

Might take awhile as docker image needs to be built

In [ ]:
deploy_model(model, mode='localhost')

### Endpoint Deployment

In [ ]:
endpoint_name = 'my-endpoint-{}'.format(int(time.time()))
deploy_model(model,
             mode='heroku',
             endpoint_name=endpoint_name)

In [ ]:
# make query to endpoint
requests.get("https://{}.herokuapp.com/endpoint/?query=i%20like%20cats".format(endpoint_name)).json()